# Linear algebra from PBSM3D Laplacian smoothing problem


In [ ]:
import numpy as np
import scipy.sparse as sparse
from scipy.io import mmread
import pandas as pd
import matplotlib.pyplot as plt
import re
import scipy.sparse.csgraph as graph
import scipy.sparse.linalg as linalg

Function to read the matrix entries from an input file. Matrices are easy to construct in "coordinate format" (COO), but generally we want a "compressed sparse X" for efficient algorithms on matrices. This function returns a compressed sparse row (CSR) matrix. 

In [ ]:
def read_vcl_matrix_file(fileName):
    df = pd.read_csv(fileName,header=None,skiprows=1,sep='\t')
    df2 = pd.DataFrame(df[0]).applymap(lambda x: x.strip())
    N = len(df2[0])
    i = np.zeros(N)
    j = np.zeros(N)
    val = np.ones(N)
    for ii in range(N):
        i[ii] = np.int64(re.findall('(\d+), (\d+)',df2[0][ii])[0][0])
        j[ii] = np.int64(re.findall('(\d+), (\d+)',df2[0][ii])[0][1])
        val[ii] = np.float(df[1][ii])
    A = sparse.coo_matrix((val,(i,j)),shape=(int(i[-1]+1),int(i[-1]+1)))
    return sparse.csr_matrix(A)

## Data download for Google Colab

Only execute this repo clone and dir change if running in Google Colab. If running on your own jupyter notebook env, you can skip ahead to "Low resolution discretization"

In [1]:
! git clone https://github.com/uofs-simlab/Math313.git
    

Cloning into 'Math313'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 9 (delta 1), reused 9 (delta 1), pack-reused 0
Unpacking objects: 100% (9/9), done.


In [ ]:
% cd Math313/20200915_direct_solves_laplacian_smoothing

## Low resolution discretization

Note that there are only 4 (at max) entries in any row. It's just the sheer scale of this thing that makes it look more dense than that.

The matrix in `lowRes_mat.out` is a finite volume discretization of $\left(I-\nabla^2\right)$

In [ ]:
%%time 
A = read_vcl_matrix_file('lowRes_mat.out')
print(A)

A function to quickly visualize the sparsity structure of matrices.

In [ ]:
def visualize_mat(A, figsize=(9,8), dpi=80, facecolor=None, edgecolor='k', markersize=0.05, title="", fontsize=20, ticklabelsize=24):
    fig=plt.figure(figsize=figsize, dpi=dpi, facecolor=facecolor, edgecolor=edgecolor)
    plot=fig.add_subplot(111)
    plot.spy(A,markersize=markersize)
    Nnz = A.nnz
    Nrow = A.shape[0]
    ratio = Nnz/(Nrow*Nrow)
    plt.title(title, fontsize=fontsize)
    plt.xlabel("Number of non-zeros: " + str(Nnz) + " (%.3f %%)" %(ratio*100),fontsize=fontsize)
    plot.tick_params(axis='both', which='major', labelsize=ticklabelsize)

In [ ]:
visualize_mat(A,title="Original order",markersize=0.1)

### Reordering the matrix

We can reorder this matrix easily using `scipy`. The reverse Cuthill--McKee (RCM) algorithm orders entries to minimize the bandwidth of the input matrix. The output is a permutation of the input indices. The permuted matrix can be obtained by symmetrically permuting the rows and columns of the original matrix.

In [ ]:
Aperm_indices = graph.reverse_cuthill_mckee(A,symmetric_mode=True)
Aperm = A[Aperm_indices,:][:,Aperm_indices]

In [ ]:
visualize_mat(Aperm,markersize=0.01,title="RCM order")

Compute the matrix bandwidth to compare the different orderings. First, write a function that can do this for the CSR storage format.

In [ ]:
def compute_bandwidth_CSR(A):
    """Compute the maximum bandwidth from a CSR sparse matrix A"""
    assert(sparse.isspmatrix_csr(A))
    indices = A.indices
    indptr = A.indptr
    bandwidth = 0
    for row in range(A.shape[0]):
        maxind = np.max(indices[indptr[row]:indptr[row+1]])
        if maxind-row > bandwidth:
            bandwidth = maxind-row
    return bandwidth

In [ ]:
print("Bandwidth of original ordering: " + str(compute_bandwidth_CSR(A)))
print("Bandwidth of RCM      ordering: " + str(compute_bandwidth_CSR(Aperm)))

### Direct factorization sparsity structures

Algorithms for sparse forward/backward substitution are $\mathcal{O}(nnz)$ where $nnz$ is the number of nonzero entries in the lower triangular factor. If a particular matrix produces a lot of fill-in upon factorization, this can be problematic both for the factorization stage (it doesn't know how much fill-in is coming, so generally ends up allocating memory many times) and the forward/backward substitution stage (there are more operations to perform for eliminations).

Start with looking at the $L$ factor in the original ordering. We should see that the number of nonzero entries is much higher than the original matrix due to fill-in.

In [ ]:
%%time
Acsc = sparse.csc_matrix(A) # LU solver needs compressed sparse column storage
LU = linalg.splu(Acsc,permc_spec="NATURAL") # NATURAL permc_spec means no pivoting
# LU = linalg.splu(Acsc,permc_spec="NATURAL",diag_pivot_thresh=0) # If an older scipy version, may need to use this one.

In [ ]:
%%time
visualize_mat(LU.L,title="Original order L")

Now look at the factorization with the RCM ordering. Observe that the time to compute this factorization is significantly lower than with the original ordering. The resulting number of nonzero entries is also substantially lower than the original ordering.

In [ ]:
%%time 
Apermcsc = sparse.csc_matrix(Aperm) # LU solver needs compressed sparse column storage
LUperm = linalg.splu(Apermcsc,options={ "Fact" : "DOFACT", "Equil" : True, "ColPerm" : "NATURAL", "Trans" : "NOTRANS", "IterRefine": "NOREFINE", 
  "DiagPivotThresh" : 0, "SymmetricMode" : True,  "RowPerm" : "NOROWPERM"})

In [ ]:
visualize_mat(LUperm.L,markersize=0.01,title="RCM order L")

### Minimum fill-in ordering

We can produce less fill-in of matrix factors by by using algorithms specifically designed to order unkowns in such a way as to minimize the amount of fill-in. Such methods are usually based around nested dissection (ND).

In `scipy`, this functionality is built into the `linalg.splu` function, and is in fact its default behaviour. Because of this, we start by looking at the factor before we look at the entire matrix.

Notice, in particular, how much faster this factorization is produced than even the RCM ordering.

In [ ]:
%%time
LUpermBEST = linalg.splu(Apermcsc)

In [ ]:
visualize_mat(LUpermBEST.L,markersize=0.01,title="ND order L")

We see that this particular ordering produces a factor with about a third the amount of entries compared to the factor generated by the RCM ordering.

The matrix corresponding to this ordering be obtained by applying the resulting permutation symmetrically (in the `.perm_c` field of the output) to the input matrix.

In [ ]:
Abest = Aperm[:,LUpermBEST.perm_c][LUpermBEST.perm_c,:]

In [ ]:
visualize_mat(Abest,markersize=0.3,title="ND order")

The resulting bandwidth of this ordering is higher than with the RCM ordering, however, we will now show that this is not a bad thing for apply sparse direct solve

In [ ]:
print("Bandwidth of ND       ordering: " + str(compute_bandwidth_CSR(Abest)))

## Direct solve times

Before starting with the solves, we construct a rhs resulting from a known solution $u^*=[1,1\ldots,1]^T$, $b=Au^*$. This allows us to evaluate the accuracy of any solve we perform. 

(NOTE that the RCM rhs needs to be permuted in accordance with the matrix rows that correspond to it. This is because its ordering was found using a method external to the `splu` method. The ND ordered RHS is the same as the RCM order RHS because the RCM matrix was used as input to `splu`)

In [ ]:
u_star = np.ones(A.shape[0])
b_orig = A*u_star
b_perm = b_orig[Aperm_indices]

Original ordering.

In [ ]:
%%time 
u_orig = LU.solve(b_orig)

RCM ordering.

In [ ]:
%%time 
u_perm = LUperm.solve(b_perm)

ND ordering.

In [ ]:
%%time 
u_best = LUpermBEST.solve(b_perm)

The ND ordered system solves more than 10 times faster on this computer. 

Finally, looking at the errors of solutions, we see that there is no substantial difference among the orderings.

In [ ]:
print("Error of original order solve: " + str(np.linalg.norm(u_star-u_orig)))
print("Error of RCM      order solve: " + str(np.linalg.norm(u_star-u_perm)))
print("Error of ND       order solve: " + str(np.linalg.norm(u_star-u_best)))